In [ ]:
from tensorflow.keras.layers import Embedding, Dense, Dropout, Bidirectional, LSTM, GRU, Input, LayerNormalization, GlobalMaxPooling1D,\
MaxPooling1D, Conv1D, LayerNormalization, Dropout,MultiHeadAttention,GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam, SGD
import tensorflow as tf
from keras.api._v2.keras import activations
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
vec_data = np.load('vectorize_maxlen_108.npy')
label_tf = np.load('label_tf.npy')

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(vec_data,label_tf,test_size = 0.2,random_state=42)

# 1.Bi LSTM

In [ ]:
def generate_model():
  dropout_thresold = 0.4
  input_dim = 4278
  output_dim = 32
  input_length = 108
  initializer = tf.keras.initializers.GlorotNormal()

  input_layers = Input(shape=(input_length))
  feature = Embedding(input_dim=input_dim, output_dim = output_dim,\
                      embeddings_initializer = 'GlorotNormal',input_length = input_length)(input_layers)

  bi_lstm_feature = Bidirectional(LSTM(units = 32, dropout = dropout_thresold,return_sequences=True,\
                                       kernel_initializer = initializer), merge_mode = 'concat')(feature)
  bi_lstm_feature = MaxPooling1D()(bi_lstm_feature)

  bi_lstm_feature = Bidirectional(GRU(units = 32, dropout = dropout_thresold,return_sequences=True,\
                                       kernel_initializer = initializer), merge_mode = 'concat')(bi_lstm_feature)
  bi_lstm_feature = MaxPooling1D()(bi_lstm_feature)
  bi_lstm_feature = LayerNormalization()(bi_lstm_feature)

  combine_feature = tf.keras.layers.Concatenate()([bi_lstm_feature])
  combine_feature = GlobalMaxPooling1D()(combine_feature)
  combine_feature = LayerNormalization()(combine_feature)

  classifier = Dense(90, activation = 'relu')(combine_feature)
  classifier = Dropout(0.2)(classifier)
  classifier = Dense(70, activation = 'relu')(classifier)
  classifier = Dropout(0.2)(classifier)
  classifier = Dense(50, activation = 'relu')(classifier)
  classifier = Dropout(0.2)(classifier)
  classifier = Dense(30, activation = 'relu')(classifier)
  classifier = Dropout(0.2)(classifier)
  classifier = Dense(3, activation = 'softmax')(classifier)

  model = tf.keras.Model(inputs = input_layers, outputs = classifier)

  return model

model_bilstm = generate_model()
adam = Adam(learning_rate = 0.001)
model_bilstm.compile(optimizer=adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
model_bilstm.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 108)]        0           []                               
                                                                                                  
 embedding_3 (Embedding)        (None, 108, 32)      136896      ['input_4[0][0]']                
                                                                                                  
 conv1d_4 (Conv1D)              (None, 108, 32)      3104        ['embedding_3[0][0]']            
                                                                                                  
 max_pooling1d_6 (MaxPooling1D)  (None, 54, 32)      0           ['conv1d_4[0][0]']               
                                                                                            

In [ ]:
callback_model = tf.keras.callbacks.ModelCheckpoint('BiLSTM_model.h5', monitors = 'val_loss')
history = model_bilstm.fit(x = X_train, y = Y_train, validation_data = (X_val,Y_val),\
                    epochs = 10, batch_size = 128, callbacks = [callback_model])

Epoch 1/10
13/13 [==============================] - 5s 359ms/step - loss: 0.2707 - accuracy: 0.9210 - val_loss: 0.4978 - val_accuracy: 0.8786
Epoch 2/10
13/13 [==============================] - 4s 310ms/step - loss: 0.2342 - accuracy: 0.9314 - val_loss: 0.5343 - val_accuracy: 0.8734
Epoch 3/10
13/13 [==============================] - 4s 316ms/step - loss: 0.2206 - accuracy: 0.9294 - val_loss: 0.5567 - val_accuracy: 0.8811
Epoch 4/10
13/13 [==============================] - 4s 310ms/step - loss: 0.1898 - accuracy: 0.9346 - val_loss: 0.5633 - val_accuracy: 0.8837
Epoch 5/10
13/13 [==============================] - 5s 375ms/step - loss: 0.1914 - accuracy: 0.9353 - val_loss: 0.5012 - val_accuracy: 0.8811
Epoch 6/10
13/13 [==============================] - 5s 404ms/step - loss: 0.2464 - accuracy: 0.9178 - val_loss: 0.4788 - val_accuracy: 0.8579
Epoch 7/10
13/13 [==============================] - 5s 425ms/step - loss: 0.1871 - accuracy: 0.9385 - val_loss: 0.5423 - val_accuracy: 0.8553
Epoch 

# 2.CNN

In [ ]:
def generate_model():
  dropout_thresold = 0.4
  input_dim = 4278
  output_dim = 32
  input_length = 108
  initializer = tf.keras.initializers.GlorotNormal()

  input_layers = Input(shape=(input_length))
  feature = Embedding(input_dim=input_dim, output_dim = output_dim,\
                      embeddings_initializer = 'GlorotNormal',input_length = input_length)(input_layers)

  cnn_feature = Conv1D(filters = 32, kernel_size = 3, padding = 'same', activation = 'relu')(feature)
  cnn_feature = MaxPooling1D()(cnn_feature)
  cnn_feature = Dropout(dropout_thresold)(cnn_feature)
  cnn_feature = Conv1D(filters = 32, kernel_size = 3, padding = 'same', activation = 'relu')(cnn_feature)
  cnn_feature = MaxPooling1D()(cnn_feature)
  cnn_feature = LayerNormalization()(cnn_feature)
  cnn_feature = Dropout(dropout_thresold)(cnn_feature)


  #combine_feature = tf.keras.layers.Concatenate()([bi_lstm_feature])
  combine_feature = GlobalMaxPooling1D()(cnn_feature)
  combine_feature = LayerNormalization()(combine_feature)

  classifier = Dense(90, activation = 'relu')(combine_feature)
  classifier = Dropout(0.2)(classifier)
  classifier = Dense(70, activation = 'relu')(classifier)
  classifier = Dropout(0.2)(classifier)
  classifier = Dense(50, activation = 'relu')(classifier)
  classifier = Dropout(0.2)(classifier)
  classifier = Dense(30, activation = 'relu')(classifier)
  classifier = Dropout(0.2)(classifier)
  classifier = Dense(3, activation = 'softmax')(classifier)

  model = tf.keras.Model(inputs = input_layers, outputs = classifier)

  return model

model_cnn = generate_model()
adam = Adam(learning_rate = 0.001)
model_cnn.compile(optimizer=adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
model_cnn.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 108)]             0         
                                                                 
 embedding_5 (Embedding)     (None, 108, 32)           136896    
                                                                 
 conv1d_8 (Conv1D)           (None, 108, 32)           3104      
                                                                 
 max_pooling1d_12 (MaxPoolin  (None, 54, 32)           0         
 g1D)                                                            
                                                                 
 dropout_28 (Dropout)        (None, 54, 32)            0         
                                                                 
 conv1d_9 (Conv1D)           (None, 54, 32)            3104      
                                                           

In [ ]:
callback_model = tf.keras.callbacks.ModelCheckpoint('CNN_model.h5', monitors = 'val_loss')
history = model_cnn.fit(x = X_train, y = Y_train, validation_data = (X_val,Y_val),\
                    epochs = 10, batch_size = 128, callbacks = [callback_model])

Epoch 1/10
13/13 [==============================] - 1s 54ms/step - loss: 0.3446 - accuracy: 0.8951 - val_loss: 0.5694 - val_accuracy: 0.8553
Epoch 2/10
13/13 [==============================] - 1s 76ms/step - loss: 0.2875 - accuracy: 0.9230 - val_loss: 0.5971 - val_accuracy: 0.8475
Epoch 3/10
13/13 [==============================] - 1s 67ms/step - loss: 0.2654 - accuracy: 0.9249 - val_loss: 0.5439 - val_accuracy: 0.8656
Epoch 4/10
13/13 [==============================] - 1s 67ms/step - loss: 0.2669 - accuracy: 0.9249 - val_loss: 0.6188 - val_accuracy: 0.8527
Epoch 5/10
13/13 [==============================] - 1s 80ms/step - loss: 0.2432 - accuracy: 0.9320 - val_loss: 0.5890 - val_accuracy: 0.8630
Epoch 6/10
13/13 [==============================] - 1s 64ms/step - loss: 0.2337 - accuracy: 0.9282 - val_loss: 0.6584 - val_accuracy: 0.8475
Epoch 7/10
13/13 [==============================] - 1s 67ms/step - loss: 0.2231 - accuracy: 0.9320 - val_loss: 0.6429 - val_accuracy: 0.8527
Epoch 8/10
13

# 3.Transformer

In [ ]:
def generate_model():
  dropout_thresold = 0.4
  input_dim = 4278
  output_dim = 32
  input_length = 108
  initializer = tf.keras.initializers.GlorotNormal()

  input_layers = Input(shape=(input_length))
  feature = Embedding(input_dim=input_dim, output_dim = output_dim,\
                      embeddings_initializer = 'GlorotNormal',input_length = input_length)(input_layers)
  trans_model = MultiHeadAttention(num_heads= 11, key_dim = 530)(feature, feature)
  trans_model = LayerNormalization()(trans_model)
  trans_model = GlobalMaxPooling1D()(trans_model)

  classifier = Dense(90, activation = 'relu')(trans_model)
  classifier = Dropout(0.2)(classifier)
  classifier = Dense(70, activation = 'relu')(classifier)
  classifier = Dropout(0.2)(classifier)
  classifier = Dense(50, activation = 'relu')(classifier)
  classifier = Dropout(0.2)(classifier)
  classifier = Dense(30, activation = 'relu')(classifier)
  classifier = Dropout(0.2)(classifier)
  classifier = Dense(3, activation = 'softmax')(classifier)

  model = tf.keras.Model(inputs = input_layers, outputs = classifier)

  return model

model_transformer = generate_model()
adam = Adam(learning_rate = 0.001)
model_transformer.compile(optimizer=adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
model_transformer.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 108)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 108, 32)      136896      ['input_1[0][0]']                
                                                                                                  
 multi_head_attention (MultiHea  (None, 108, 32)     560319      ['embedding[0][0]',              
 dAttention)                                                      'embedding[0][0]']              
                                                                                                  
 layer_normalization (LayerNorm  (None, 108, 32)     64          ['multi_head_attention[0][0]'

In [ ]:
callback_model = tf.keras.callbacks.ModelCheckpoint('Transformer_model.h5', monitors = 'val_loss')
history = model_transformer.fit(x = X_train, y = Y_train, validation_data = (X_val,Y_val),\
                    epochs = 10, batch_size = 128, callbacks = [callback_model])

Epoch 1/10
13/13 [==============================] - 404s 21s/step - loss: 0.9701 - accuracy: 0.4505 - val_loss: 0.8688 - val_accuracy: 0.5478
Epoch 2/10
13/13 [==============================] - 132s 9s/step - loss: 0.9079 - accuracy: 0.4725 - val_loss: 0.8647 - val_accuracy: 0.4496
Epoch 3/10
13/13 [==============================] - 87s 7s/step - loss: 0.8820 - accuracy: 0.5113 - val_loss: 0.8182 - val_accuracy: 0.6460
Epoch 4/10
13/13 [==============================] - 83s 6s/step - loss: 0.8673 - accuracy: 0.5625 - val_loss: 0.7951 - val_accuracy: 0.6873
Epoch 5/10
13/13 [==============================] - 72s 6s/step - loss: 0.7766 - accuracy: 0.6427 - val_loss: 0.6644 - val_accuracy: 0.7571
Epoch 6/10
13/13 [==============================] - 74s 6s/step - loss: 0.6226 - accuracy: 0.7929 - val_loss: 0.5129 - val_accuracy: 0.8320
Epoch 7/10
13/13 [==============================] - 74s 6s/step - loss: 0.5171 - accuracy: 0.8362 - val_loss: 0.4790 - val_accuracy: 0.8424
Epoch 8/10
13/13 